# Distribución de cargas simulado con algoritmo genetico con número enteros


In [49]:
from algoritmo_genetico import AlgoritmoGenetico

In [50]:
class DivisionCargasAGBin(AlgoritmoGenetico):
    def __init__(self, ventana: list, num_procesadores: int, underload: int, overload: int, low_val=0, top_val=1, num_genes=20, num_cromosomas=50, num_generaciones=200, prob_cruce=0.8, prob_mutacion=0.05, porcentaje_elitismo=0.05):
        super().__init__(low_val, top_val, num_cromosomas, num_genes,
                         num_generaciones, prob_cruce, prob_mutacion, porcentaje_elitismo)
        self.ventana = ventana
        self.num_procesadores = num_procesadores
        self.underload = underload
        self.overload = overload
        self.len_bin = len(bin(self.num_procesadores-1)[2:])
        self.num_genes *= self.len_bin

    def evaluar_fitness(self, cromosoma: list):
        carga_por_procesador = [0] * self.num_procesadores

        j = 0
        for proceso in self.ventana:
            list_bin = cromosoma[j:j+self.len_bin]
            procesador = int("".join(map(str, list_bin)), 2)
            j += self.len_bin
            carga_por_procesador[procesador] += proceso

        self.carga_maxima = max(carga_por_procesador)
        cargas_aceptadas = sum(1 for carga in carga_por_procesador if self.underload < carga < self.overload)
        promedio_carga = sum(carga / self.carga_maxima for carga in carga_por_procesador) / self.num_procesadores

        return (1.0 / self.carga_maxima) * promedio_carga * (cargas_aceptadas / self.num_procesadores)
    
    def best(self, poblacion: list):
        return min(poblacion, key=self.evaluar_fitness)


In [51]:
def generar_division_cargas_AGBin (n_procesos:int, n_procesadores:int, size_ventana:int, ventana:list):
    overloaded = (n_procesos // n_procesadores) + 100
    residuo = n_procesos % n_procesadores
    if residuo > 0:
        ventana.append(residuo)
        
    return  DivisionCargasAGBin(
    ventana = ventana,
    num_procesadores = n_procesadores, 
    underload = 50, overload = overloaded,
    num_genes = size_ventana, num_cromosomas = 100)

## Parametros


In [52]:
n_procesos = 3600
size_ventana = 54
ventana = [n_procesos // size_ventana] * size_ventana
top_pow = 5
cargas_ag = {}
maximos = {}
poblaciones = {}

## Implementacion

In [53]:
for i in range(1, top_pow):
    n_procesadores = 2 ** i
    cargas_ag[i] = generar_division_cargas_AGBin(n_procesos = n_procesos, n_procesadores = n_procesadores, size_ventana = size_ventana, ventana = ventana)
    poblaciones[i] = cargas_ag[i].fit()    
    maximos[i] = cargas_ag[i].carga_maxima


## Resultados

In [54]:
for i in range(1, top_pow):
    print(f"Distribución de cargas {2 ** i} procesadores: ")
    print("Tiempo total : ", cargas_ag[i].carga_maxima)

    best = cargas_ag[i].best(poblaciones[i])
    print("Mejor solución: ", end = "")
    for j in range(0, len(best), i):
        grupo = best[j:j+i]
        print(grupo, end = " ")
        
    print("\nFitness: ", cargas_ag[i].evaluar_fitness(best))
    print("------------------------------------------------------------------------")



Distribución de cargas 2 procesadores: 
Tiempo total :  1914
Mejor solución: [0] [0] [1] [1] [0] [1] [1] [1] [1] [1] [1] [1] [1] [1] [0] [0] [1] [0] [1] [1] [1] [1] [1] [1] [1] [1] [1] [1] [0] [0] [0] [1] [1] [1] [0] [1] [1] [0] [0] [0] [1] [0] [0] [1] [1] [1] [1] [1] [0] [0] [0] [1] [1] [0] 
Fitness:  0.00016697588126159552
------------------------------------------------------------------------
Distribución de cargas 4 procesadores: 
Tiempo total :  1056
Mejor solución: [1, 1] [0, 1] [1, 1] [1, 0] [0, 0] [0, 0] [0, 0] [1, 1] [1, 1] [0, 0] [0, 0] [0, 0] [1, 0] [1, 0] [0, 0] [0, 0] [1, 1] [1, 1] [0, 0] [1, 1] [0, 1] [1, 1] [0, 0] [1, 1] [0, 0] [0, 0] [0, 0] [0, 0] [0, 0] [1, 0] [0, 1] [0, 0] [1, 1] [0, 0] [1, 0] [0, 0] [0, 0] [1, 1] [1, 0] [0, 0] [1, 0] [1, 0] [1, 0] [0, 0] [0, 0] [0, 1] [0, 0] [1, 1] [1, 0] [1, 0] [1, 0] [0, 0] [0, 0] [0, 0] 
Fitness:  0.00022693652501344812
------------------------------------------------------------------------
Distribución de cargas 8 procesadores:

## Graficas